In [1]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
from google.cloud import storage
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import glob
import os

In [5]:
sc.setLogLevel("ERROR")

from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np

In [25]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [6]:
client = storage.Client()
bucket_name = 'my-project-bucket-jl'
bucket = client.get_bucket(bucket_name)
elements = bucket.list_blobs()
csv_files=[a.name for a in elements]

In [7]:
mlb_gameday_cleaned_list = []
for file in csv_files:
    filename = 'gs://my-project-bucket-jl/' + file + ''
    word = 'revised'
    if word not in filename: 
        continue 
    else: 
        df = filename
        mlb_gameday_cleaned_list.append(df)

In [8]:
mlb_gameday_list = []
for file in mlb_gameday_cleaned_list:
    try:
        filename = file 
        df = pd.read_parquet(filename)
        if df.empty:
            continue
        else:
            filename = file 
            print(filename)
            df = spark.read.parquet(filename)
            mlb_gameday_list.append(df)
    except pd.errors.EmptyDataError:
        continue

gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2005_04


gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2005_05


gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2005_06
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2005_07
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2005_08
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2005_09
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2005_10
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2006_04
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2006_05
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2006_06
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2006_07
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2006_08
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2006_09
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2006_10
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2007_03

gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2018_10
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2019_03
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2019_04
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2019_05
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2019_06
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2019_07
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2019_08
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2019_09
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2019_10
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2020_07
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2020_08
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2020_09
gs://my-project-bucket-jl/cleaned/revised_mlb_gameday_pitch_sequence_2020_10

In [9]:
merged_mlb_gameday_list_df = None
for df in mlb_gameday_list: 
    if merged_mlb_gameday_list_df == None: 
        merged_mlb_gameday_list_df = df 
    else: 
        merged_mlb_gameday_list_df = merged_mlb_gameday_list_df.union(df)

In [10]:
new_merged_mlb_gameday_list_df = merged_mlb_gameday_list_df.fillna(0)

In [11]:
got_on_base = ['single','home_run','double','walk',
               'hit_by_pitch','triple','catcher_interf',
               'fielders_choice','field_error','intent_walk']
at_bat_continues = [None,'truncated_pa','injury',
                    'ejection','game_advisory']
out = ['field_out','strikeout','sac_fly','grounded_into_double_play',
       'force_out','sac_bunt','fielders_choice_out','double_play',
       'strikeout_double_play','triple_play', 'sac_fly_double_play', 
       'sac_bunt_double_play','strikeout_triple_play']
def events_outcome(x):
    if x in at_bat_continues: 
        return 1
    elif x in out:
        return 0
    elif x in got_on_base:
        return 2
events_outcome_udf = udf(events_outcome)

In [12]:
new_merged_mlb_gameday_list_df = new_merged_mlb_gameday_list_df.withColumn('event_outcome',events_outcome_udf('events'))

In [13]:
new_merged_mlb_gameday_list_df = new_merged_mlb_gameday_list_df.withColumn("player_on_1b", when(new_merged_mlb_gameday_list_df.on_1b == 0.0, 0.0).otherwise(1.0))

In [14]:
new_merged_mlb_gameday_list_df = new_merged_mlb_gameday_list_df.withColumn("player_on_2b", when(new_merged_mlb_gameday_list_df.on_1b == 0.0, 0.0).otherwise(1.0))

In [15]:
new_merged_mlb_gameday_list_df = new_merged_mlb_gameday_list_df.withColumn("player_on_3b", when(new_merged_mlb_gameday_list_df.on_1b == 0.0, 0.0).otherwise(1.0))

In [16]:
new_merged_mlb_gameday_list_df = new_merged_mlb_gameday_list_df.withColumn("label",when(new_merged_mlb_gameday_list_df.pitch_type == "FF", 1.0).otherwise(0.0) )

In [17]:
transformed_sdf = new_merged_mlb_gameday_list_df.sample(False, .01)

In [18]:
trainingData, testData = transformed_sdf.randomSplit([0.75, 0.25], seed=42)

In [19]:
indexer = StringIndexer(inputCols=["batter", "pitcher",'stand','p_throws','type',
                                   'inning_topbot','game_pk','event_outcome',
                                   'game_year'], 
                        outputCols=["batterIndex", 
                                    "pitcherIndex",'standIndex','p_throwsIndex',
                                    'typeIndex','inning_topbotIndex','game_pkIndex',
                                    'event_outcomeIndex','game_yearIndex'],
                       handleInvalid= "skip")

In [20]:
encoder = OneHotEncoder(inputCols=["batterIndex","pitcherIndex",'standIndex',
                                   'p_throwsIndex','typeIndex',
                                    'inning_topbotIndex','game_pkIndex',
                                    'event_outcomeIndex','game_yearIndex'],
                        outputCols=["batterVector","pitcherVector",'standVector',
                                    'p_throwsVector','typeVector',
                                    'inning_topbotVector','game_pkVector',
                                    'event_outcomeVector','game_yearVector'],
                        dropLast=False)

In [21]:
assembler = VectorAssembler(inputCols=["batterVector","pitcherVector",'standVector',
                                       'p_throwsVector','typeVector','inning_topbotVector',
                                       'game_pkVector','event_outcomeVector','game_yearVector',
                                       'zone','balls', 'strikes','outs_when_up','inning',
                                       'at_bat_number','pitch_number','home_score','away_score',
                                       'post_away_score','post_home_score','player_on_1b',
                                       'player_on_2b','player_on_3b'], 
                            outputCol="features")

In [22]:
#Random Forrest Classifier
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)
rf_pipeline = Pipeline(stages=[indexer,encoder,assembler,rf])

In [23]:
pitch_type_pipe = Pipeline(stages=[indexer, encoder, assembler])


In [24]:
new_transformed_sdf = pitch_type_pipe.fit(transformed_sdf).transform(transformed_sdf)


24/12/13 22:24:32 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
24/12/13 22:25:10 WARN DAGScheduler: Broadcasting large task binary with size 1110.1 KiB


In [26]:
#hyperparameter grid
grid = ParamGridBuilder().addGrid(rf.numTrees, [10, 30]).addGrid(rf.maxDepth, [5, 10, 20]).build()

In [27]:
print('Number of models to be tested: ', len(grid))

Number of models to be tested:  6


In [28]:
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")

In [29]:
cv = CrossValidator(estimator=rf_pipeline,
                    estimatorParamMaps=grid,
                    evaluator=evaluator,
                    numFolds=2) 

In [30]:
cv = cv.fit(trainingData)

24/12/13 22:25:47 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/12/13 22:26:02 WARN DAGScheduler: Broadcasting large task binary with size 7.1 MiB
24/12/13 22:26:30 WARN DAGScheduler: Broadcasting large task binary with size 7.2 MiB
24/12/13 22:26:38 WARN DAGScheduler: Broadcasting large task binary with size 7.2 MiB
24/12/13 22:26:41 WARN DAGScheduler: Broadcasting large task binary with size 7.5 MiB
24/12/13 22:26:49 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/12/13 22:26:50 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/12/13 22:26:51 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/12/13 22:26:53 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/12/13 22:27:02 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/12/13 22:27:11 WARN DAGScheduler: 

24/12/13 22:54:08 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/12/13 22:55:07 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/12/13 22:56:12 WARN DAGScheduler: Broadcasting large task binary with size 9.0 MiB
24/12/13 22:57:19 WARN DAGScheduler: Broadcasting large task binary with size 8.6 MiB
24/12/13 22:57:28 WARN DAGScheduler: Broadcasting large task binary with size 7.2 MiB
24/12/13 22:57:34 WARN DAGScheduler: Broadcasting large task binary with size 7.2 MiB
24/12/13 22:57:37 WARN DAGScheduler: Broadcasting large task binary with size 7.5 MiB
24/12/13 22:57:42 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/12/13 22:57:43 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/12/13 22:57:43 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/12/13 22:57:44 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/12/13 22:57:51 WARN DAGScheduler: Broadcasting larg

24/12/13 23:40:38 WARN DAGScheduler: Broadcasting large task binary with size 7.2 MiB
24/12/13 23:40:40 WARN DAGScheduler: Broadcasting large task binary with size 7.5 MiB
24/12/13 23:40:46 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/12/13 23:40:46 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/12/13 23:40:46 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/12/13 23:40:52 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/12/13 23:40:59 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB
24/12/13 23:41:28 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB
24/12/13 23:42:05 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/12/13 23:42:54 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/12/13 23:43:39 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/12/13 23:44:32 WARN DAGScheduler: Broadcasting larg

In [32]:
bestModel = cv.bestModel

In [37]:
feature_importances = bestModel.stages[-1].featureImportances

In [47]:
predictions = cv.transform(testData)


In [48]:
var_index = dict()
for variable_type in ['numeric', 'binary']:
    for variable in predictions.schema["features"].metadata["ml_attr"]["attrs"][variable_type]:
         idx = variable['idx']
         name = variable['name']
         var_index[idx] = name 

In [49]:
for i in range(len(var_index)):
    print(f"Feature Importance {i} {var_index[i]}  {feature_importances[i]}")


Feature Importance 0 batterVector_458015.0  5.3477160298516075e-05
Feature Importance 1 batterVector_429665.0  0.0
Feature Importance 2 batterVector_502671.0  0.0
Feature Importance 3 batterVector_457708.0  0.0003958623252409591
Feature Importance 4 batterVector_460086.0  0.0
Feature Importance 5 batterVector_434158.0  4.93376939810104e-05
Feature Importance 6 batterVector_457705.0  0.0025807498429139945
Feature Importance 7 batterVector_429664.0  0.0
Feature Importance 8 batterVector_134181.0  0.0008994116575454658
Feature Importance 9 batterVector_408234.0  0.00023132054632756782
Feature Importance 10 batterVector_431145.0  0.0
Feature Importance 11 batterVector_467793.0  0.0
Feature Importance 12 batterVector_407893.0  0.0
Feature Importance 13 batterVector_443558.0  0.0003065097003376818
Feature Importance 14 batterVector_608070.0  0.0002705626471576704
Feature Importance 15 batterVector_452678.0  0.0
Feature Importance 16 batterVector_408045.0  0.0
Feature Importance 17 batterVect